In [6]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [7]:
def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

In [8]:
embed1 = bert_embedding('barcode scanner')
embed_verb = bert_embedding('scan')
embed2 = bert_embedding('product')

In [9]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

In [10]:
records, summary, key = driver.execute_query("""
MATCH (n:Entity)-[r:RELATED_TO]->(m:Entity)
WITH n.name as subject, m.name as object, r.type as verb, vector.similarity.cosine(n.embedding, $embed_one) as subjectScore, 
    vector.similarity.cosine(m.embedding, $embed_two) as objectScore,
    vector.similarity.cosine(r.embedding, $embed_verb) as verbScore
RETURN subject, object, verb, (subjectScore + objectScore + verbScore) / 3 as score
ORDER BY score DESC LIMIT 5
""", embed_one=embed1, embed_two=embed2, embed_verb=embed_verb)

for row in records:
    print(row['subject'], row['verb'], row['object'], row['score'])

discount tags are used for product 0.9948160847028097
product set allows products 0.9939572215080261
payment process is more efficient 0.9932372172673544
fiscal positions enables rules 0.9926049510637919
Odoo Point of Sale allows Multi Employees per Session 0.992323080698649
